In [126]:
import subprocess
import pandas as pd
import os
import pickle
import seaborn as sns
#base_directory = '/central/scratch/tbellagi/gea/baypass_terminal/cmd_files/shfiles/'

In [73]:
base_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/baypass_terminal/cmd_files/shfiles/'

In [74]:
##the bfffiles have the bayes factor for each of the covariates for each fo the markers 
bffiles = [i for i in os.listdir('cmd_files/shfiles/') if 'betai_reg.out' in i]
#xtxfiles = [i for i in os.listdir('results') if 'pi_xtx.out' in i]

In [75]:
bffiles

['partition_101_chain_3_summary_betai_reg.out',
 'partition_100_chain_2_summary_betai_reg.out',
 'partition_0_chain_1_summary_betai_reg.out',
 'partition_103_chain_1_summary_betai_reg.out',
 'partition_105_chain_1_summary_betai_reg.out',
 'partition_100_chain_3_summary_betai_reg.out',
 'partition_101_chain_2_summary_betai_reg.out',
 'partition_102_chain_1_summary_betai_reg.out',
 'partition_104_chain_1_summary_betai_reg.out',
 'partition_0_chain_3_summary_betai_reg.out',
 'partition_102_chain_2_summary_betai_reg.out',
 'partition_103_chain_3_summary_betai_reg.out',
 'partition_101_chain_1_summary_betai_reg.out',
 'partition_104_chain_2_summary_betai_reg.out',
 'partition_103_chain_2_summary_betai_reg.out',
 'partition_0_chain_2_summary_betai_reg.out',
 'partition_102_chain_3_summary_betai_reg.out',
 'partition_100_chain_1_summary_betai_reg.out',
 'partition_105_chain_2_summary_betai_reg.out',
 'partition_104_chain_3_summary_betai_reg.out']

In [76]:
## caltech 
#base_directory = '/central/scratch/tbellagi/gea/baypass_terminal/cmd_files/shfiles/'

In [77]:
env_vars_names = ['bio1', 'bio17']

In [80]:

base_directory = '/carnegie/nobackup/scratch/tbellagio/gea_grene-net/baypass_terminal/cmd_files/shfiles/'
##the bfffiles have the bayes factor for each of the covariates for each fo the markers 
bffiles = [i for i in os.listdir('cmd_files/shfiles/') if 'betai_reg.out' in i]
#xtxfiles = [i for i in os.listdir('results') if 'pi_xtx.out' in i]

len(bffiles)

def extract_partitions(bffiles):
    partitions = {}
    for file_name in bffiles:
        parts = file_name.split('_')
        partition = parts[1]
        chain = parts[3]
        if partition not in partitions:
            partitions[partition] = []
        partitions[partition].append(chain)
    return partitions

file_pattern = 'partition_{partition}_chain_{chain}_summary_betai_reg.out'

def process_file(partition, chain, output_dir):
    bf_file_name = file_pattern.format(partition=partition, chain=chain)
    bf_file = pd.read_csv(base_directory + bf_file_name, sep='\s+')
    
    # Load loci and environment variable names
    partition_name = 'partition_{}'.format(partition)
    with open(f'individual_gfiles/loci_{partition_name}', 'rb') as f:
        gloci = pickle.load(f)
    gdict = {num+1: locus for num, locus in enumerate(gloci)}
    bf_file['MRK'] = bf_file['MRK'].map(gdict)
    
    with open('env_vars_names', 'rb') as f:
        env_vars_names = pickle.load(f)
    env_vars_dict = {num+1: env_var for num, env_var in enumerate(env_vars_names)}
    bf_file['COVARIABLE'] = bf_file['COVARIABLE'].map(env_vars_dict)

    # Rename columns to include chain number, except for 'COVARIABLE' and 'MRK'
    bf_file = bf_file.rename(columns={col: f"{col}_chain{chain}" for col in bf_file.columns if col not in ['COVARIABLE', 'MRK']})

    # Write to file by COVARIABLE
    for covariable, df_group in bf_file.groupby('COVARIABLE'):
        output_path = os.path.join(output_dir, f"combined_covariable_{covariable}_chain{chain}.csv")
        df_group.drop(columns='COVARIABLE').to_csv(output_path, mode='a', index=False, header=not os.path.exists(output_path))


partitions = extract_partitions(bffiles)  # Assuming bffiles is defined somewhere

all_part_collected = list(partitions.keys())

output_directory = 'output_test/'
os.makedirs(output_directory, exist_ok=True)

for partition, chains in partitions.items():
    for chain in chains:
        process_file(partition, chain, output_directory)

In [132]:
chain1_bio1 = pd.read_csv('output/combined_covariable_bio1_chain1.csv')#['chain'].unique()

In [142]:
chain1_bio1

,MRK,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1
0,2_1140846,0.020074,0.057350,0.050055,0.050678,-23.010300,0.000000e+00,0.000000,0.012022
1,2_1140854,0.015487,0.061344,0.035964,0.055086,-23.010300,0.000000e+00,0.000000,0.024496
2,2_1140895,0.006409,0.049780,0.000626,0.047879,-23.006425,-2.480000e-06,0.000012,0.077525
3,2_1140909,0.020843,0.046658,0.002683,0.053217,-23.002330,-1.990000e-06,0.000036,0.019717
4,2_1141426,-0.003370,0.058337,0.020751,0.052782,-23.010300,0.000000e+00,0.000000,0.005132
...,...,...,...,...,...,...,...,...,...
1038904,1_9974783,-0.023122,0.043767,0.076668,0.041129,-23.009647,-3.200000e-07,0.000002,0.062421
1038905,1_9974845,-0.009992,0.048184,0.008095,0.040096,-23.009818,-3.200000e-07,0.000002,0.070028
1038906,1_9975089,-0.041689,0.044152,0.017375,0.045115,-22.725061,-1.147200e-04,0.000318,0.143695
1038907,1_9975134,-0.035745,0.067121,-0.034927,0.047487,-22.700269,-5.050000e-05,0.000285,0.065882


In [133]:
len(chain1_bio1)

1038909

In [134]:
chain2_bio1 = pd.read_csv('output/combined_covariable_bio1_chain2.csv')#['chain'].unique()

In [135]:
len(chain2_bio1)

1044132

In [136]:
chain3_bio1 = pd.read_csv('output/combined_covariable_bio1_chain3.csv')#['chain'].unique()

In [137]:
len(chain3_bio1)

1038909

In [138]:
bio1_results = chain1_bio1.merge(chain2_bio1, on = 'MRK').merge(chain3_bio1, on = 'MRK')

In [145]:
def rank_baypass(df):
    """Create columns to rank loci within and across chains."""
    import pandas
    import tqdm
    import math
    chains = ['chain1', 'chain2', 'chain3']  # Your chains are named 'chain1', 'chain2', 'chain3'

    # rank per chain based on chain BF
    for chain in tqdm.tqdm(chains, desc='chains'):
        col = f'BF(dB)_{chain}'  # Adjust column names to match your DataFrame
        colBF = df[col]  # Use the 'BF(dB)' columns as-is
        ranked = dict((locus, rank+1) for (rank, locus) in enumerate(colBF.sort_values(ascending=False).index))
        df[f'rank_BF_{chain}'] = [ranked[locus] for locus in df.index]  # Adding the rank column for each chain

    # mean BF across chains
    print('getting mean BF')
    bfcols = [f'BF(dB)_{chain}' for chain in chains]  # List of BF columns across chains
    df['mean_BF(dB)'] = df[bfcols].mean(axis=1)  # Mean of BF values across chains

    # rank of mean BF
    print('ranking mean BF')
    ranked = dict((locus, rank+1) for (rank, locus) in enumerate(df['mean_BF(dB)'].sort_values(ascending=False).index))
    df['rank_mean_BF(dB)'] = [ranked[locus] for locus in df.index]  # Rank based on the mean BF

    # bool column: BF >= 20 for >=3 chains, and BF >= 15 for >= 3 chains
    print('calculating bool column')
    df['BF(dB)_gte20_for-gte3chains'] = (df[bfcols] >= 1).sum(axis=1) >= 3
    df['BF(dB)_gte15_for-gte3chains'] = (df[bfcols] >= 1).sum(axis=1) >= 3

    return df

# Example of calling the function


In [139]:
bio1_results[bio1_results['BF(dB)_chain3'] > 0]

,MRK,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1,M_Pearson_chain2,...,SD_Beta_is_chain2,eBPis_chain2,M_Pearson_chain3,SD_Pearson_chain3,M_Spearman_chain3,SD_Spearman_chain3,BF(dB)_chain3,Beta_is_chain3,SD_Beta_is_chain3,eBPis_chain3
8033,5_8336003,0.022850,0.056431,0.020282,0.042958,-22.944670,0.000029,0.000169,0.063705,0.022869,...,0.000100,0.071881,0.011468,0.082156,0.017458,0.060223,2.247637,0.000175,0.000646,0.104445
12897,1_19210413,0.012357,0.049720,-0.068654,0.057476,-23.002133,0.000004,0.000024,0.065878,0.060974,...,0.000727,0.152368,-0.035441,0.079695,-0.039874,0.052873,5.559726,-0.000375,0.000963,0.156582
15157,1_19406005,0.020432,0.060756,0.005342,0.044947,-22.771437,0.000121,0.000271,0.183023,-0.022437,...,0.000055,0.040020,0.054826,0.094571,-0.010832,0.047191,1.140802,0.000827,0.001238,0.297371
15765,4_9740070,-0.001903,0.052671,0.001463,0.052238,-23.007016,0.000001,0.000006,0.086224,-0.021289,...,0.000038,0.095241,0.021783,0.079735,-0.010338,0.048463,8.091739,0.000203,0.000699,0.112577
21655,4_112123,-0.021634,0.058859,-0.039208,0.042856,-22.873526,-0.000015,0.000206,0.026541,0.043533,...,0.001085,0.239626,-0.034491,0.084426,-0.025847,0.037955,4.028885,-0.000531,0.001073,0.207134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001243,3_22373730,0.024609,0.057994,0.009051,0.034972,-22.992861,-0.000005,0.000027,0.072936,0.036392,...,0.000232,0.096931,-0.082222,0.081154,0.041475,0.032930,10.491120,-0.000782,0.001231,0.279838
1005470,1_21068541,0.011381,0.050650,0.046005,0.042281,-22.996890,0.000008,0.000032,0.101918,0.032873,...,0.000074,0.152607,-0.075654,0.090500,0.054439,0.044945,0.248885,-0.000833,0.001249,0.296938
1017775,1_11068086,0.019316,0.045099,0.050027,0.043279,-22.939460,0.000041,0.000130,0.123239,0.015006,...,0.000021,0.075311,0.032202,0.073923,0.067918,0.045246,1.437504,0.000294,0.000799,0.146887
1027348,4_13875769,0.001762,0.061359,-0.023305,0.055847,-22.976173,0.000021,0.000070,0.114710,-0.012608,...,0.000022,0.049082,0.061465,0.102089,-0.013048,0.058622,15.466926,0.000691,0.001191,0.250320


In [ ]:
bio1_results[bio1_results['BF(dB)_chain2'] > 0]

In [ ]:
bio1_results[bio1_results['BF(dB)_chain1'] > 0]

In [141]:
bio1_results[(bio1_results['BF(dB)_chain1'] > 0) & (bio1_results['BF(dB)_chain2'] > 0) & (bio1_results['BF(dB)_chain3'] > 0)]

,MRK,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1,M_Pearson_chain2,...,SD_Beta_is_chain2,eBPis_chain2,M_Pearson_chain3,SD_Pearson_chain3,M_Spearman_chain3,SD_Spearman_chain3,BF(dB)_chain3,Beta_is_chain3,SD_Beta_is_chain3,eBPis_chain3


In [140]:
sns.histplot(bio1_results['BF(dB)_chain3'].sample(1000))

<Axes: xlabel='BF(dB)_chain3', ylabel='Count'>

Error in callback <function flush_figures at 0x7fd21d1ee670> (for post_execute):



KeyboardInterrupt



In [ ]:
sns.histplot(bio1_results['BF(dB)_chain3'])

In [151]:
bio1_results

,MRK,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1,M_Pearson_chain2,...,Beta_is_chain3,SD_Beta_is_chain3,eBPis_chain3,rank_BF_chain1,rank_BF_chain2,rank_BF_chain3,mean_BF(dB),rank_mean_BF(dB),BF(dB)_gte20_for-gte3chains,BF(dB)_gte15_for-gte3chains
0,2_1140846,0.020074,0.057350,0.050055,0.050678,-23.010300,0.000000e+00,0.000000,0.012022,0.014486,...,0.000000e+00,0.000000,0.017156,677137,676991,761936,-23.010300,799899,False,False
1,2_1140854,0.015487,0.061344,0.035964,0.055086,-23.010300,0.000000e+00,0.000000,0.024496,-0.001352,...,0.000000e+00,0.000000,0.013512,538113,1033445,1032793,-23.010300,603814,False,False
2,2_1140895,0.006409,0.049780,0.000626,0.047879,-23.006425,-2.480000e-06,0.000012,0.077525,0.008470,...,-4.870000e-06,0.000033,0.053467,252776,217775,212865,-23.004093,276286,False,False
3,2_1140909,0.020843,0.046658,0.002683,0.053217,-23.002330,-1.990000e-06,0.000036,0.019717,0.002017,...,-8.000000e-08,0.000001,0.026099,210900,268683,361738,-23.006590,302389,False,False
4,2_1141426,-0.003370,0.058337,0.020751,0.052782,-23.010300,0.000000e+00,0.000000,0.005132,0.012218,...,0.000000e+00,0.000000,0.013226,852496,852451,852532,-23.010300,877839,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033681,1_9974783,-0.023122,0.043767,0.076668,0.041129,-23.009647,-3.200000e-07,0.000002,0.062421,-0.003887,...,-1.240000e-06,0.000012,0.038994,334281,300004,289578,-23.008951,345841,False,False
1033682,1_9974845,-0.009992,0.048184,0.008095,0.040096,-23.009818,-3.200000e-07,0.000002,0.070028,-0.000791,...,8.490000e-06,0.000056,0.055383,345513,397280,174128,-23.005288,287502,False,False
1033683,1_9975089,-0.041689,0.044152,0.017375,0.045115,-22.725061,-1.147200e-04,0.000318,0.143695,-0.035581,...,-1.200000e-06,0.000018,0.024156,38211,23542,215547,-22.714094,56649,False,False
1033684,1_9975134,-0.035745,0.067121,-0.034927,0.047487,-22.700269,-5.050000e-05,0.000285,0.065882,-0.003634,...,5.000000e-07,0.000003,0.058988,36166,382344,330334,-22.906663,104198,False,False


In [147]:
bio1_results_bf = rank_baypass(bio1_results)

chains: 100%|██████████| 3/3 [00:03<00:00,  1.13s/it]


getting mean BF
ranking mean BF
calculating bool column


In [153]:
bio1_results_bf[bio1_results_bf['BF(dB)_gte20_for-gte3chains']==True]

,MRK,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1,M_Pearson_chain2,...,Beta_is_chain3,SD_Beta_is_chain3,eBPis_chain3,rank_BF_chain1,rank_BF_chain2,rank_BF_chain3,mean_BF(dB),rank_mean_BF(dB),BF(dB)_gte20_for-gte3chains,BF(dB)_gte15_for-gte3chains


In [150]:
bio1_results_bf[bio1_results_bf['mean_BF(dB)'] > 0 ]

,MRK,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1,M_Pearson_chain2,...,Beta_is_chain3,SD_Beta_is_chain3,eBPis_chain3,rank_BF_chain1,rank_BF_chain2,rank_BF_chain3,mean_BF(dB),rank_mean_BF(dB),BF(dB)_gte20_for-gte3chains,BF(dB)_gte15_for-gte3chains
466858,4_6671462,0.008125,0.04413,0.055765,0.044786,-23.005716,-8.300000e-07,0.000008,0.0368,0.015007,...,0.000614,0.001118,0.23473,243378,86921,1,4.487865,1,False,False


In [123]:
bio1_results[bio1_results['BF(dB)_gte20_for-gte3chains'] == True]

,MRK,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1,M_Pearson_chain2,...,Beta_is_chain3,SD_Beta_is_chain3,eBPis_chain3,rank_BF_chain1,rank_BF_chain2,rank_BF_chain3,mean_BF(dB),rank_mean_BF(dB),BF(dB)_gte20_for-gte3chains,BF(dB)_gte15_for-gte3chains


In [111]:
import math

# Calculate the number of rows corresponding to the top 1% of ranked SNPs
oneperc = math.ceil(0.01 * bio1_results.shape[0])  # bio1_results.shape[0] gives the number of rows

# Identify the columns that contain the ranking information for each chain
rankcols = [col for col in bio1_results.columns if 'rank_chain' in col]

# Create a new column for rank consistency: True if ranked in the top 1% for >= 3 chains
bio1_results['rank_consistency_top1perc_for-gte3chains'] = (bio1_results[rankcols] < oneperc).sum(axis=1) >= 3


In [112]:
bio1_results

,MRK,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1,M_Pearson_chain2,...,SD_Beta_is_chain3,eBPis_chain3,rank_BF_chain1,rank_BF_chain2,rank_BF_chain3,mean_BF(dB),rank_mean_BF(dB),BF(dB)_gte20_for-gte3chains,BF(dB)_gte15_for-gte3chains,rank_consistency_top1perc_for-gte3chains
0,3_10219089,0.007446,0.036700,-0.003083,0.039417,-22.993054,9.920000e-06,0.000031,0.124425,-0.027338,...,0.000032,0.021757,5193,5460,4364,-22.991486,6659,False,False,False
1,3_10219212,-0.008280,0.055622,-0.062376,0.052075,-22.997813,6.090000e-06,0.000025,0.093788,-0.010901,...,0.000031,0.128330,5899,7037,4912,-22.997692,7548,False,False,False
2,3_10219218,0.004010,0.047771,0.004848,0.045716,-23.010300,0.000000e+00,0.000000,0.010526,-0.024760,...,0.000000,0.013175,25639,15844,25606,-23.010300,17515,False,False,False
3,3_10219278,0.000185,0.058097,-0.056077,0.046092,-22.985299,-9.700000e-07,0.000059,0.005766,0.002059,...,0.000013,0.045647,4416,6791,6847,-22.997002,7427,False,False,False
4,3_10219281,-0.002439,0.060129,-0.068364,0.045447,-22.830401,8.537000e-05,0.000246,0.137278,0.005691,...,0.000600,0.132933,1520,6620,194,-21.736845,563,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31268,3_12322089,0.024506,0.063186,0.010761,0.034821,-22.937187,3.242000e-05,0.000174,0.069341,0.004071,...,0.000083,0.031797,2568,11731,5114,-22.980294,5665,False,False,False
31269,3_12322273,0.004862,0.055868,-0.006902,0.051856,-23.008106,1.480000e-06,0.000010,0.057084,-0.009252,...,0.000008,0.028626,9062,12473,10179,-23.009214,11158,False,False,False
31270,3_12322285,0.018958,0.051278,-0.009422,0.048508,-23.010300,0.000000e+00,0.000000,0.012215,-0.005987,...,0.000000,0.005968,19319,20009,18025,-23.010300,20117,False,False,False
31271,3_12322677,0.016151,0.057352,0.019272,0.049998,-23.010300,0.000000e+00,0.000000,0.012031,-0.004008,...,0.000000,0.011575,19040,20011,19548,-23.010300,20935,False,False,False


In [39]:
shfiles = []

file = f'baypass_postproc.sh'
text = f'''#!/bin/bash
#SBATCH --job-name=baypass_postproc
#SBATCH --time=48:00:00  
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=40gb
#SBATCH --cpus-per-task=1
#SBATCH --output=baypass_postproc.out
#SBATCH --mail-user=tbellagio@carnegiescience.edu
#SBATCH --mail-type=FAIL

# Source the conda.sh script
source /home/tbellagio/miniforge3/etc/profile.d/conda.sh

# Activate the conda environment
conda activate /home/tbellagio/miniforge3/envs/pipeline_snakemake

cd /carnegie/nobackup/scratch/tbellagio/gea_grene-net/baypass_terminal/
python read_results.py 
    
    
    '''
with open(file, 'w') as o:
    o.write("%s" % text)
shfiles.append(file)

In [40]:
subprocess.run(["sbatch", shfiles[0]], check=True)

Submitted batch job 7590


CompletedProcess(args=['sbatch', 'baypass_postproc.sh'], returncode=0)

In [9]:
bio1_results = pd.read_csv('output/combined_covariable_bio1.csv')

In [11]:
len(bio1_results)/3

1040650.0

In [12]:
bio1_results

,MRK,M_Pearson_chain3,SD_Pearson_chain3,M_Spearman_chain3,SD_Spearman_chain3,BF(dB)_chain3,Beta_is_chain3,SD_Beta_is_chain3,eBPis_chain3
0,2_1140846,0.008693,0.059067,0.040813,0.047365,-23.010300,0.000000e+00,0.000000,0.017156
1,2_1140854,-0.000336,0.058375,0.025435,0.057003,-23.010300,0.000000e+00,0.000000,0.013512
2,2_1140895,-0.009405,0.059067,-0.004854,0.047786,-23.002636,-4.870000e-06,0.000033,0.053467
3,2_1140909,0.002350,0.054464,0.003107,0.051158,-23.010001,-8.000000e-08,0.000001,0.026099
4,2_1141426,0.013986,0.053166,0.014386,0.044028,-23.010300,0.000000e+00,0.000000,0.013226
...,...,...,...,...,...,...,...,...,...
3121945,1_9974783,-0.002669,0.051964,0.064243,0.051500,-23.008411,-1.240000e-06,0.000012,0.038994
3121946,1_9974845,0.025551,0.044766,0.029413,0.044755,-22.995839,8.490000e-06,0.000056,0.055383
3121947,1_9975089,0.016319,0.038861,0.035461,0.049187,-23.002968,-1.200000e-06,0.000018,0.024156
3121948,1_9975134,0.010919,0.049597,0.001917,0.045669,-23.009573,5.000000e-07,0.000003,0.058988


In [16]:
pd.read_csv('output/old/combined_covariable_bio1.csv', nrows=10)

,MRK,M_Pearson_chain3,SD_Pearson_chain3,M_Spearman_chain3,SD_Spearman_chain3,BF(dB)_chain3,Beta_is_chain3,SD_Beta_is_chain3,eBPis_chain3,M_Pearson_chain5,...,SD_Beta_is_chain4,eBPis_chain4,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1
0,5_26905969,0.030803,0.044185,-0.014937,0.026941,-16.381096,0.001709,0.002517,0.303618,0.011378,...,0.003883,0.618614,0.023559,0.063010,-0.020037,0.035662,-14.856028,0.001891,0.003055,0.271003
1,5_26905997,0.008035,0.013821,0.054238,0.016376,-17.355942,0.000601,0.001092,0.235061,0.070906,...,0.001734,0.605315,0.012142,0.015186,0.068327,0.012816,-17.337486,0.001012,0.001157,0.418543
2,5_26906001,-0.013106,0.015917,-0.049088,0.020593,-19.731181,-0.000489,0.000700,0.314491,0.016696,...,0.001732,0.125461,-0.032300,0.023222,-0.044769,0.018022,-18.945756,-0.001388,0.000999,0.783642
3,5_26906090,-0.029118,0.032199,-0.015904,0.028848,-18.073204,-0.001533,0.001535,0.497974,-0.010222,...,0.002752,1.917697,-0.017336,0.036041,-0.041965,0.032886,-18.838927,-0.000764,0.001323,0.249128
4,5_26906130,0.164371,0.034921,0.010976,0.027337,14.091749,0.011975,0.003734,2.872791,0.154100,...,0.005669,1.386075,0.219567,0.050575,0.048391,0.021426,34.053432,0.013609,0.004948,2.225036
5,5_26906135,0.175780,0.028407,0.031800,0.028172,12.815528,0.012272,0.003392,3.528008,0.290552,...,0.005256,1.507637,0.181330,0.046809,0.011527,0.018734,18.804166,0.012896,0.004693,2.221858
6,5_26906226,-0.053451,0.101745,-0.023851,0.035743,3.817387,-0.005478,0.007906,0.311251,-0.073004,...,0.005886,0.468021,-0.114260,0.074972,0.014125,0.039653,14.994689,-0.007733,0.006734,0.600572
7,5_26906227,-0.085903,0.066428,-0.048570,0.037556,11.038768,-0.006293,0.006769,0.452799,-0.091773,...,0.007162,0.751844,-0.100390,0.094010,-0.001326,0.046929,10.514507,-0.007339,0.007712,0.466870
8,5_26906331,-0.049656,0.053121,0.002526,0.024397,-19.899410,-0.000606,0.000852,0.321701,-0.034739,...,0.000689,0.208180,-0.035688,0.078198,-0.048621,0.041386,-20.986478,-0.000661,0.000812,0.381011
9,5_26906341,0.156062,0.017427,0.031650,0.030211,2.990140,0.009576,0.001251,13.719061,0.167329,...,0.000618,15.352530,0.171359,0.018357,0.030474,0.010678,6.313716,0.010419,0.001250,15.352530


In [46]:
ex = ex.drop_duplicates()

In [47]:
ex

,MRK,M_Pearson_chain1,SD_Pearson_chain1,M_Spearman_chain1,SD_Spearman_chain1,BF(dB)_chain1,Beta_is_chain1,SD_Beta_is_chain1,eBPis_chain1
0,3_18787786,-0.057994,0.022873,-0.022699,0.056887,-15.383687,-0.003277,0.001207,2.178231
1,3_18787884,0.095886,0.015284,0.083745,0.026692,-10.826386,0.005984,0.001021,8.343154
2,3_18787966,0.055884,0.105654,0.075887,0.047679,15.121666,0.003579,0.005935,0.262388
3,3_18788130,-0.004774,0.041041,0.020203,0.037783,-20.926410,-0.000471,0.000821,0.247206
4,3_18788146,-0.034610,0.029371,0.015065,0.045917,-17.000865,-0.002072,0.001670,0.668053
...,...,...,...,...,...,...,...,...,...
85225,1_8509837,-0.009847,0.059982,0.062447,0.035828,-15.174250,-0.000599,0.001432,0.170185
85226,1_8509841,-0.007103,0.038802,0.051701,0.042173,-20.033965,0.000017,0.001138,0.005054
85227,1_8509845,0.008897,0.040864,0.007956,0.038057,-18.024636,0.000200,0.002160,0.033209
85228,1_8509959,0.000952,0.044789,0.068739,0.050719,-21.424707,-0.000155,0.000807,0.071801


In [71]:
bf_file = pd.read_csv('results/' + bffiles[0], sep = '\s+')#.to_csv('chat.csv') #['COVARIABLE']

In [72]:
gdict = dict((num+1, locus) for num,locus in enumerate(gloci))


In [75]:
bf_file['MRK'] = bf_file['MRK'].map(gdict)

In [76]:
bf_file

,COVARIABLE,MRK,M_Pearson,SD_Pearson,M_Spearman,SD_Spearman,BF(dB),Beta_is,SD_Beta_is,eBPis
0,1,5_26905969,0.030803,0.044185,-0.014937,0.026941,-16.381096,0.001709,0.002517,0.303618
1,1,5_26905997,0.008035,0.013821,0.054238,0.016376,-17.355942,0.000601,0.001092,0.235061
2,1,5_26906001,-0.013106,0.015917,-0.049088,0.020593,-19.731181,-0.000489,0.000700,0.314491
3,1,5_26906090,-0.029118,0.032199,-0.015904,0.028848,-18.073204,-0.001533,0.001535,0.497974
4,1,5_26906130,0.164371,0.034921,0.010976,0.027337,14.091749,0.011975,0.003734,2.872791
...,...,...,...,...,...,...,...,...,...,...
323,4,5_26908558,-0.084529,0.072838,0.011450,0.049190,-22.247325,-0.000239,0.000548,0.178339
324,4,5_26908660,0.068867,0.053512,0.039562,0.041900,-11.654302,0.001182,0.001300,0.439961
325,4,5_26908842,-0.015835,0.060358,-0.017711,0.052363,-22.879418,-0.000051,0.000227,0.085005
326,4,5_26908852,-0.014924,0.040654,-0.036039,0.047861,-23.006084,-0.000003,0.000012,0.087723


In [46]:
partitions

0    partition_1680
1    partition_1680
2    partition_1664
3    partition_1664
4    partition_1680
5    partition_1680
6    partition_1664
7    partition_1664
8    partition_1680
9    partition_1664
dtype: object

In [ ]:
def label_snps(resfile):
    """Label baypass resfiles with SNP ID using pkl files saved in 002_kickoff."""
    import os
    import pandas
    import tqdm
    # find the gfile associated with resfile
    prefix = os.path.basename(resfile).split("_chain")[0]
    chain = "chain_%s" % os.path.basename(resfile).split(prefix)[1].split("_")[2]
    
    # get the pkl file that has the ordered list of SNP IDs for the gfile
    if 'new' in resfile:
        newgfile_dir = os.path.dirname(resfile).replace("new_results", "new_individual_gfiles")
        pklfile = os.path.join(newgfile_dir, f'{prefix}_noheaderidx.pkl')
    else:
        pklfile = os.path.join(gfile_dir, f'{prefix}_noheaderidx.pkl')
    gloci = pklload(pklfile)  # read in rows with SNPIDs, but only one column
    
    # read in the resfile
    print('reading table ...')
    df = pandas.read_table(resfile, delim_whitespace=True)
    
    # map MRK column (SNP index) to locus name
    print('getting gdict ...')
    gdict = dict((num+1, locus) for num,locus in enumerate(gloci))
    
    # map covariable column to environment ID
    print('getting edict ...')
    edict = dict((num+1, env) for num,env in enumerate(efile.index))
    
    # split each covariable (env) in to its own file, label index with SNP ID
    df['env'] = df['COVARIABLE'].map(edict)
    envdfs = {chain:{}}
    for env in tqdm.tqdm(uni(df['env']), desc='annotating snps'):
        envdfs[chain][env] = df[df['env']==env].copy()
        envdfs[chain][env].index = envdfs[chain][env]['MRK'].map(gdict)
        envdfs[chain][env].index.names = ['']

    return envdfs

efile = pd.read_table(op.join(baydir, 'coastal_efile_std_HEADERIDX.txt'), index_col=0, usecols=[0,1])
dview['efile'] = efile
dview['uni'] = uni
dview['gfile_dir'] = gfile_dir
dview['pklload'] = pklload

In [21]:
import subprocess

def grep_search(directory, search_text):
    try:
        # Command to execute grep
        command = ['grep', '-rl', search_text, directory]
        # Running the command
        result = subprocess.run(command, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        # Printing the result
        if result.stdout:
            print("Files containing the search term:")
            print(result.stdout)
        else:
            print("No files found with the search term.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Usage
grep_search('cmd_files/shfiles/', 'partition_1680_chain_3')


Files containing the search term:
cmd_files/shfiles/batch_102.sh



In [22]:
# Open the file
with open('cmd_files/shfiles/' + 'batch_102.sh', 'r') as file:
    content = file.read()

In [23]:
content

'#!/bin/bash\n#SBATCH --job-name=batch_102\n#SBATCH --time=4-00:00:00\n#SBATCH --nodes=1\n#SBATCH --ntasks=1\n#SBATCH --mem-per-cpu=10gb\n#SBATCH --cpus-per-task=8\n#SBATCH --output=batch_102_%j.out\n#SBATCH --mail-user=tbellagio@carnegiescience.edu\n#SBATCH --mail-type=FAIL\n\ncd /home/tbellagi/bigscratch/gea/baypass_terminal/results/\n/home/tbellagi/bin/baypass -gfile /home/tbellagi/bigscratch/gea/baypass_terminal/individual_gfiles/partition_1680.txt -efile /home/tbellagi/bigscratch/gea/baypass_terminal/env_3rdgen_nheader.txt -omegafile /home/tbellagi/bigscratch/gea/baypass_terminal/omegaavg_nheader.txt -poolsizefile /home/tbellagi/bigscratch/gea/baypass_terminal/pool_sizes_3rdgen_nheader.txt -outprefix partition_1680_chain_3 -d0yij 0.4 -seed 17632 -pilotlength 1000 -nval 50000 -nthreads 8 -npilot 40 \n'

In [25]:
pd.read_csv('individual_gfiles/' + 'partition_1680.txt',header=None)

,0,1,2,3,4,5,6,7,8,9,...,700,701,702,703,704,705,706,707,708,709
0,51,143,61,56,1,23,16,4,28,44,...,8,16,1,6,7,14,2,18,5,15
1,12,182,3,114,0,24,1,19,7,65,...,2,22,1,6,4,17,7,13,4,16
2,7,187,4,113,0,24,0,20,13,59,...,1,23,0,7,0,21,0,20,0,20
3,160,34,108,9,21,3,20,0,61,11,...,23,1,7,0,20,1,18,2,18,2
4,11,183,9,108,1,23,0,20,17,55,...,10,14,0,7,2,19,1,19,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,6,188,6,111,0,24,0,20,0,72,...,0,24,0,7,0,21,0,20,0,20
78,39,155,16,101,16,8,1,19,14,58,...,2,22,1,6,1,20,6,14,6,14
79,6,188,2,115,0,24,2,18,2,70,...,0,24,0,7,3,18,0,20,0,20
80,6,188,2,115,0,24,2,18,2,70,...,0,24,0,7,3,18,0,20,0,20


In [30]:
len(data_loaded)

82

In [31]:
data_loaded

['5_26905969',
 '5_26905997',
 '5_26906001',
 '5_26906090',
 '5_26906130',
 '5_26906135',
 '5_26906226',
 '5_26906227',
 '5_26906331',
 '5_26906341',
 '5_26906413',
 '5_26906450',
 '5_26906487',
 '5_26906591',
 '5_26906602',
 '5_26906603',
 '5_26906662',
 '5_26906663',
 '5_26906698',
 '5_26906730',
 '5_26906794',
 '5_26906815',
 '5_26906854',
 '5_26906900',
 '5_26906922',
 '5_26907018',
 '5_26907137',
 '5_26907194',
 '5_26907199',
 '5_26907240',
 '5_26907254',
 '5_26907274',
 '5_26907309',
 '5_26907335',
 '5_26907438',
 '5_26907473',
 '5_26907478',
 '5_26907482',
 '5_26907503',
 '5_26907675',
 '5_26907682',
 '5_26907683',
 '5_26907721',
 '5_26907724',
 '5_26907727',
 '5_26907773',
 '5_26907775',
 '5_26907777',
 '5_26907781',
 '5_26907782',
 '5_26907793',
 '5_26907798',
 '5_26907893',
 '5_26907920',
 '5_26907921',
 '5_26907947',
 '5_26907963',
 '5_26907966',
 '5_26907993',
 '5_26908007',
 '5_26908014',
 '5_26908098',
 '5_26908103',
 '5_26908136',
 '5_26908227',
 '5_26908242',
 '5_269082